# Web scrapping de letras de músicas
Neste notebook acessamos a plataforma [Letras](https://letras.mus.br) usando web scrapping e salvamos de um determinado artista em uma tabela, permitindo uma análise posterior.

In [46]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import os, sys
from unicodedata import normalize

# Escolha do artista

In [30]:
website = 'https://www.letras.mus.br/'
# o nome do artista deve estar como escrito na sua página de letras.mus.br. E.g.: luiz-gonzaga
artist = 'belchior'
url = website + artist
# condição para só incluir as composições originais do artista selecionado. 
# Deve ser usada como False caso deseje onter letras de interprétes
original_composer = True

# Parsing dos links de todas as letras

In [8]:
# acessa a página do artista
try:
        req = requests.get(url)
except requests.exceptions.RequestException as e:
        print(e) 
             
# obtém o conteúdo da página   
content = req.content
soup = BeautifulSoup(content, 'html.parser')
soup = BeautifulSoup(content, 'html.parser')
soup.title

<title>Belchior - LETRAS.MUS.BR</title>

In [11]:
# obtém elementos html das letras
lyrics_html = soup.find_all("a", {"class": "song-name"})

# processa elementos transformando em string
str_lyrics = str(lyrics_html)
#href="/belchior/959402/"
split = 'href="/' + artist + '/'
lyrics = str_lyrics.split(split)
lyrics = lyrics[1:]
print('Quantidade de letras:',len(lyrics))

# parsing para obter os links a partir dos elementos html
lyrics_links = []
for link in lyrics:
    link = link.split('/')
    link = link[0]
    link = website + artist + '/' + link + '/'
    lyrics_links.append(link)
    print(link)


Quantidade de letras: 128
https://www.letras.mus.br/belchior/a-cor-do-cacau/
https://www.letras.mus.br/belchior/44448/
https://www.letras.mus.br/belchior/344900/
https://www.letras.mus.br/belchior/1154802/
https://www.letras.mus.br/belchior/239370/
https://www.letras.mus.br/belchior/239371/
https://www.letras.mus.br/belchior/153384/
https://www.letras.mus.br/belchior/1839470/
https://www.letras.mus.br/belchior/400596/
https://www.letras.mus.br/belchior/344901/
https://www.letras.mus.br/belchior/1538319/
https://www.letras.mus.br/belchior/44449/
https://www.letras.mus.br/belchior/400597/
https://www.letras.mus.br/belchior/344902/
https://www.letras.mus.br/belchior/411035/
https://www.letras.mus.br/belchior/709644/
https://www.letras.mus.br/belchior/bahiuno/
https://www.letras.mus.br/belchior/350408/
https://www.letras.mus.br/belchior/296787/
https://www.letras.mus.br/belchior/400598/
https://www.letras.mus.br/belchior/1060059/
https://www.letras.mus.br/belchior/1391391/
https://www.letr

# Seleciona as músicas mais acessadas

In [15]:
# músicas mais acessadas
most_played = soup.find_all("ol", {"class": "cnt-list cnt-list--num -flex-col-2 js-song-list"})
str_most_played = str(most_played)
#href="/belchior/959402/"
list_most_played = str_most_played.split(split)
list_most_played = list_most_played[1:]
print('Quantidade de letras mais acessadas:',len(list_most_played))

links_most_played = []
for link in list_most_played:
    link = link.split('/')
    link = link[0]
    link = website + artist + '/' + link + '/'
    links_most_played.append(link)
    print(link)

Quantidade de letras mais acessadas: 20
https://www.letras.mus.br/belchior/153384/
https://www.letras.mus.br/belchior/44448/
https://www.letras.mus.br/belchior/44453/
https://www.letras.mus.br/belchior/344922/
https://www.letras.mus.br/belchior/44449/
https://www.letras.mus.br/belchior/44462/
https://www.letras.mus.br/belchior/44451/
https://www.letras.mus.br/belchior/44454/
https://www.letras.mus.br/belchior/saia-do-meu-caminho/
https://www.letras.mus.br/belchior/44464/
https://www.letras.mus.br/belchior/44459/
https://www.letras.mus.br/belchior/132598/
https://www.letras.mus.br/belchior/44457/
https://www.letras.mus.br/belchior/1448417/
https://www.letras.mus.br/belchior/44452/
https://www.letras.mus.br/belchior/44460/
https://www.letras.mus.br/belchior/44456/
https://www.letras.mus.br/belchior/44458/
https://www.letras.mus.br/belchior/132699/
https://www.letras.mus.br/belchior/44463/


# Recuperar letra a partir de um link

In [37]:
def clean_html(raw_html):
    CLEANR = re.compile('<.*?>') 
    clean_text = re.sub(CLEANR, ' ', raw_html)
    clean_text = re.sub(' +', ' ', clean_text)
    return clean_text

def parse_lyric(url, links_most_played = None):
    """
    Recebe o link de uma letra e, opcionalmente, uma lista com os links das músicas mais tocadas.
    Retorna titulo, compositor, primeiro compositor, se é mais tocada e o texto da letra
    """
    # testa se é uma das músicas mais tocadas
    most_played = False
    if links_most_played and url in links_most_played:
        most_played = True        
    # acessa o link da letra
    try:
        req = requests.get(url)
    except requests.exceptions.RequestException as e: 
        print(e)        
    content = req.content
    soup = BeautifulSoup(content, 'html.parser')
    
    #compositor
    #<div class="letra-info_comp">
    html_comp = soup.find_all("div", {"class": "letra-info_comp"})
    html_comp = str(html_comp)
    try:
        composer = html_comp.split('Composição: ')[1]
        composer = composer.split('<a')[0]
        composer = composer[0:-2].strip()
    except:
        composer = 'Não identificado'
    first_composer = composer.split(' / ')[0] 
    
    
    #título da música
    #<div class="cnt-head_title">
    html_title = soup.find_all("div", {"class": "cnt-head_title"})
    html_title = str(html_title)
    title = html_title.split('<h1>')[1]
    title = title.split('</h1>')[0]
    
    #letra da música
    html = soup.find_all("div", {"class": "cnt-letra p402_premium"})
    html = str(html)
    verses = html.split('<p>')
    verses = verses[1:]

    lyric = ''
    for verse in verses:
        # remove tags html        
        verse = clean_html(verse)
        lyric += verse

    lyric = lyric.strip()
    return title, composer, first_composer, most_played, lyric

In [38]:
parse_lyric('https://www.letras.mus.br/elis-regina/45670/')

('Como Nossos Pais',
 'Belchior',
 'Belchior',
 False,
 'Não quero lhe falar, meu grande amor De coisas que aprendi nos discos Quero lhe contar como eu vivi E tudo o que aconteceu comigo Viver é melhor que sonhar Eu sei que o amor é uma coisa boa Mas também sei que qualquer canto É menor do que a vida de qualquer pessoa Por isso cuidado, meu bem Há perigo na esquina Eles venceram e o sinal está fechado pra nós Que somos jovens Para abraçar seu irmão E beijar sua menina na rua É que se fez o seu braço O seu lábio e a sua voz Você me pergunta pela minha paixão Digo que estou encantada como uma nova invenção Eu vou ficar nesta cidade, não vou voltar pro sertão Pois vejo vir vindo no vento cheiro de nova estação Eu sei de tudo na ferida viva do meu coração Já faz tempo, eu vi você na rua Cabelo ao vento, gente jovem reunida Na parede da memória Essa lembrança é o quadro que dói mais Minha dor é perceber Que apesar de termos feito tudo o que fizemos Ainda somos os mesmos e vivemos Ainda som

# Acessa todos os links e armazena as informações

In [22]:
def normalization(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

In [39]:
lyrics_list = []
most_played_list = []
titles = []
composer_target = artist.replace('-',' ')
composer_list = []

for url in lyrics_links:
    title, composer, first_composer, most_played, lyric = parse_lyric(url)    
    print(title)
    print(composer)
    print(most_played)
    print(lyric)
    print()
    
    composer_normalized = composer.lower()
    composer_normalized = normalization(composer)
    first_composer = first_composer.lower()
    first_composer = normalization(first_composer)  

    # só adciona ao dataframe se o compositor estiver definido na página e for igual ao artista
    # algumas letras podem não ter compositor definido. pode-se remover essa condição, nesses casos
    # também testa se já existe alguma letra com o mesmo título no dataset
    # se houver mais de um compositor, testará se o primeiro é igual ao artista buscado
    if original_composer:
        if (composer == composer_target or first_composer == composer_target) and (title not in titles):
            lyrics_list.append(lyric)
            most_played_list.append(most_played)
            titles.append(title)
            composer_list.append(composer)
            
    # inclui todas as letras
    else:
        lyrics_list.append(lyric)
        most_played_list.append(most_played)
        titles.append(title)
        composer_list.append(composer)

A Cor do Cacau
Guilherme Arantes
False
A minha morena é cor do cacau Sabor de banana é Sol tropical Este balanço é pra te embalar Esta canção é pra te encantar Estes acordes são recordação Que nunca é tarde, não, no coração O meu Reggae é pra regalar A minha Salsa é pra temperar O meu Calipso é pra refrescar Ah, e o Merengue é pra suspirar ]

A Palo Seco
Belchior
False
Se você vier me perguntar por onde andei No tempo em que você sonhava De olhos abertos, lhe direi Amigo, eu me desesperava Sei que assim falando pensas Que esse desespero é moda em 76 Mas ando mesmo descontente Desesperadamente, eu grito em português Mas ando mesmo descontente Desesperadamente, eu grito em português Tenho vinte e cinco anos De sonho e de sangue E de América do Sul Por força deste destino Um tango argentino Me vai bem melhor que um blues Sei que assim falando pensas Que esse desespero é moda em 76 E eu quero é que esse canto torto Feito faca, corte a carne de vocês E eu quero é que esse canto torto Feito 

# Criando e salvando DataFrame em CSV

In [42]:
df = pd.DataFrame(data = {'titulo': titles, 
                         'mais tocada': most_played_list,
                         'letra': lyrics_list,
                         'compositor': composer_list})

In [44]:
df.head()

,titulo,mais tocada,letra,compositor
0,A Palo Seco,False,Se você vier me perguntar por onde andei No te...,Belchior
1,Aguapé,False,Capineiro de meu pai Não me cortes meus cabelo...,Belchior / CASTRO ALVES / Raimundo Fagner Epig...
2,Alucinação,False,Eu não estou interessado em nenhuma teoria Em ...,Belchior
3,Amor e Crime,False,"Amor, não há amor Existem só provas de amor Ma...",Belchior / francisco Casaverde
4,Apenas Um Rapaz Latino Americano,False,Eu sou apenas um rapaz latino-americano Sem di...,Belchior


In [48]:
# insert dir of python scripts in the path
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path

'c:\\Users\\lucas.carneiro\\visualizando_belchior'

In [50]:
filename = artist + '.csv'
filepath = os.path.join(module_path, 'raw_data', filename)
filepath

'c:\\Users\\lucas.carneiro\\visualizando_belchior\\raw_data\\belchior.csv'

In [52]:
df.to_csv(filepath, index = False)